In [14]:
import pandas as pd
import os
from datetime import datetime, timedelta
import talib
import pandas as pd
import numpy as np
import cufflinks as cf
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import plotly.graph_objs as go
import warnings
import enum
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True) 
import sqlite3
import ipywidgets as widgets
from IPython import display
from IPython.display import clear_output
from tabulate import tabulate

In [15]:
class StatisticsManager:

    def __init__(self,name):    
        self.name =name  
        self.db = sqlite3.connect(self.name)
        

    def connectDB(self):
        self.db = sqlite3.connect(self.name)
        
    def get_trade_all(self):
        self.connectDB()
        query = '''SELECT * FROM Statistics'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_day(self, day):
        self.connectDB()
        query = '''SELECT s.close_time, sum(case when  s.status = 'loss' then 1 else 0 end) as loss,
                       sum(case when s.status = 'profit' then 1 else 0 end) as profit,
                       round(sum(case when  s.status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when s.status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(case when s.type = 'long' then s.close*s.number_of_shares -s.current*s.number_of_shares else s.current*s.number_of_shares -s.close*s.number_of_shares end),2) as turnover,
                       round(1.5*(select count(*) from Statistics t WHERE t.close_time = s.close_time),2) as tax,
                       round(sum(case when s.type = 'long' then s.close*s.number_of_shares -s.current*s.number_of_shares else s.current*s.number_of_shares -s.close*s.number_of_shares end)
                       - 1.5*(select count(*) from Statistics t WHERE t.close_time = s.close_time),2) as returns
                    FROM Statistics s GROUP by s.close_time having s.close_time = '''
        query += "'" + day + "'"
        query += ''' ORDER by s.close_time'''
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data

    def get_trade_loss(self):
        self.connectDB()
        query = '''SELECT s.close_time, sum(case when  s.status = 'loss' then 1 else 0 end) as loss,
                       sum(case when s.status = 'profit' then 1 else 0 end) as profit,
                       sum(case when s.status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  s.status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when s.status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(case when s.type = 'long' then s.close*s.number_of_shares -s.current*s.number_of_shares else s.current*s.number_of_shares -s.close*s.number_of_shares end),2) as turnover,
                       round(1.5*(select count(*) from Statistics t WHERE t.close_time = s.close_time),2) as tax,
                       round(sum(case when s.type = 'long' then s.close*s.number_of_shares -s.current*s.number_of_shares else s.current*s.number_of_shares -s.close*s.number_of_shares end)
                       - 1.5*(select count(*) from Statistics t WHERE t.close_time = s.close_time),2) as returns
                    FROM Statistics s GROUP by s.close_time having returns < 0  ORDER by profit_ratio DESC'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_win(self):
        self.connectDB()
        query = '''SELECT algorithm,close_time, sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       sum(case when status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(close*number_of_shares -current*number_of_shares ),2)  as returns
                    FROM Statistics GROUP by close_time having returns > 0  ORDER by returns'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    

    def get_trade_close_time(self):
        self.connectDB()
        query = '''SELECT s.close_time, sum(case when  s.status = 'loss' then 1 else 0 end) as loss,
                       sum(case when s.status = 'profit' then 1 else 0 end) as profit,
                       round(sum(case when  s.status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when s.status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(case when s.type = 'long' then s.close*s.number_of_shares -s.current*s.number_of_shares else s.current*s.number_of_shares -s.close*s.number_of_shares end),2) as turnover,
                       round(1.5*(select count(*) from Statistics t WHERE t.close_time = s.close_time),2) as tax,
                       round(sum(case when s.type = 'long' then s.close*s.number_of_shares -s.current*s.number_of_shares else s.current*s.number_of_shares -s.close*s.number_of_shares end)
                       - 1.5*(select count(*) from Statistics t WHERE t.close_time = s.close_time),2) as returns
                    FROM Statistics s GROUP by s.close_time ORDER by s.close_time'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_algorithm(self):
        self.connectDB()
        query = '''SELECT algorithm,sum(case when  status = 'loss' then 1 else 0 end) as loss,
                       sum(case when status = 'profit' then 1 else 0 end) as profit,
                       round(sum(case when  status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(case when type = 'long' then close*number_of_shares -current*number_of_shares else current*number_of_shares -close*number_of_shares end),2)turnover,
                       round(1.5*(select count(*) from Statistics),2) as tax,
                       round(sum(case when type = 'long' then close*number_of_shares -current*number_of_shares else current*number_of_shares -close*number_of_shares end)
                       - 1.5*(select count(*) from Statistics) ,2) as returns
                    FROM Statistics GROUP by algorithm ORDER by profit_ratio desc'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data
    
    def get_trade_symbol(self):
        self.connectDB()
        query = '''SELECT s.symbol,s.type, sum(case when  s.status = 'loss' then 1 else 0 end) as loss,
                       sum(case when s.status = 'profit' then 1 else 0 end) as profit,
                       sum(case when s.status = 'open' then 1 else 0 end) as open,
                       round(sum(case when  s.status = 'loss' then 1.0 else 0.0 end)/count(*),2) as loss_ratio,
                       round(sum(case when s.status = 'profit' then 1.0 else 0.0 end)/count(*),2)  as profit_ratio,
                       round(sum(case when s.type = 'long' then s.close*s.number_of_shares -s.current*s.number_of_shares else s.current*s.number_of_shares -s.close*s.number_of_shares end),2) as turnover,
                       round(1.5*(select count(*) from Statistics t WHERE t.symbol = s.symbol),2) as tax,
                       round(sum(case when s.type = 'long' then s.close*s.number_of_shares -s.current*s.number_of_shares else s.current*s.number_of_shares -s.close*s.number_of_shares end)
                       - 1.5*(select count(*) from Statistics t WHERE t.symbol = s.symbol),2) as returns
                    FROM Statistics s GROUP by s.symbol,s.type  ORDER by returns'''        
        data = pd.read_sql(query, con=self.db)
        self.close()
        return data


    def close(self):
        self.db.close()

    def deleteDB(self,db):
        if os.path.exists(db):
            os.remove(db)
        else:
            print("The file does not exist")

In [16]:
def visualize_returns(data,title):
    fig = px.bar(data, x='close_time', y='returns',
             hover_data=['turnover', 'tax','returns'], color='returns',
                 title=title,
             labels={'date':'close_time'}, height=400)
    fig.show()

In [17]:
def showTable(data,index=False):
    print(tabulate(data, headers='keys', tablefmt='psql',showindex=index))

In [18]:
nikhil_db_initial = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\nikhil\\nikhil_intial.db"
nikhil_db_v1 = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\nikhil\\nikhil_v1.db"
nikhil_db_v2 = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\nikhil\\nikhil_v2.db"
nikhil_current = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\db\\nikhil\\statistics.db"


In [22]:
db_list = [nikhil_db_initial,nikhil_db_v1,nikhil_db_v2,nikhil_current]
algorithm_data = pd.DataFrame(columns = ['algorithm', 'loss', 'profit','loss_ratio','profit_ratio','turnover'])
manager = StatisticsManager(nikhil_db_initial)
manager.get_trade_algorithm()
for db in db_list:
    manager.name = db
    row_data = manager.get_trade_algorithm()
    algorithm_data = algorithm_data.append(row_data, ignore_index=True)
algorithm_data
showTable(algorithm_data)

+-------------+--------+----------+--------------+----------------+------------+----------+-----------+
| algorithm   |   loss |   profit |   loss_ratio |   profit_ratio |   turnover |      tax |   returns |
|-------------+--------+----------+--------------+----------------+------------+----------+-----------|
| Nikhil      |  46316 |    48503 |         0.49 |           0.51 |    68254   | 142228   |  -73974.5 |
| Nikhil      |  24225 |    27790 |         0.47 |           0.53 |   100307   |  78022.5 |   22284.8 |
| Nikhil      |  25087 |    28923 |         0.46 |           0.54 |   102939   |  81015   |   21924.2 |
| Nikhil      |   1315 |     1598 |         0.45 |           0.55 |     7573.2 |   4369.5 |    3203.7 |
+-------------+--------+----------+--------------+----------------+------------+----------+-----------+


In [23]:
manager = StatisticsManager(nikhil_current)
current_data = manager.get_trade_close_time()
current_data['close_time'] = current_data['close_time'].apply(pd.to_datetime).dt.date.astype(str)
visualize_returns(current_data,"nikhil_current")

manager = StatisticsManager(nikhil_db_v2)
v2_data = manager.get_trade_close_time()
v2_data['close_time'] = v2_data['close_time'].apply(pd.to_datetime).dt.date.astype(str)
visualize_returns(v2_data,"nikhil_db_v2")

manager = StatisticsManager(nikhil_db_v1)
v1_data = manager.get_trade_close_time()
v1_data['close_time'] = v1_data['close_time'].apply(pd.to_datetime).dt.date.astype(str)
visualize_returns(v1_data,"nikhil_db_v1")



